In [1]:
import os
import os.path
import numpy as np
import h5py
import time
import matplotlib.pyplot as plt

import skimage.io as sio

/users/visics/kkelchte/tensorflow/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
# Define which folders to parse:
root_dir='/esat/opal/kkelchte/docker_home/pilot_data/esat'
# root_dir='/esat/opal/kkelchte/docker_home/pilot_data/small'
runs=sorted([root_dir+'/'+d for d in os.listdir(root_dir) if d.startswith('0')])
print len(runs),' runs.'
print [os.path.basename(r) for r in runs]

25  runs.
['00000_esat', '00001_esat', '00002_esat', '00003_esat', '00004_esat', '00005_esat', '00006_esat', '00007_esat', '00008_esat', '00009_esat', '00010_esat', '00011_esat', '00012_esat', '00013_esat', '00014_esat', '00015_esat', '00016_esat', '00017_esat', '00018_esat', '00019_esat', '00020_esat', '00021_esat', '00022_esat', '00023_esat', '00024_esat']


In [9]:
root_name= 'data.hdf5'
# remove if it already exists
if os.path.isfile(root_dir+'/'+root_name):
    os.remove(root_dir+'/'+root_name)
f = h5py.File(root_dir+'/'+root_name, 'w')

In [10]:
# create a group for this run
for r in runs: f.create_group(os.path.basename(r))

In [11]:
def add_dir(group, data_type='RGB'):
    if os.path.isdir(r+'/'+data_type):
        # get an idea of the shape
        image_files=sorted([r+'/'+data_type+'/'+i for i in os.listdir(r+'/'+data_type)])
        img=sio.imread(image_files[0])
        try:
            img_array=np.zeros((len(image_files), img.shape[0], img.shape[1], img.shape[2]),dtype=np.uint8)
        except:
            img_array=np.zeros((len(image_files), img.shape[0], img.shape[1]),dtype=np.uint8)
        for i, imgf in enumerate(image_files): img_array[i]=sio.imread(imgf)
#         plt.imshow(img_array[-1])
#         plt.show()
        print('extracted {0}, shape: {1}'.format(data_type,img_array.shape))
        return group.create_dataset(data_type,data=img_array, compression="gzip", compression_opts=8)
#         return group.create_dataset(data_type,data=img_array, compression="lzf")
    else:
        print('could not find {0} in {1}.'.format(data_type,group.name))
        return []


In [12]:
def add_file(group, file_name='control_info'):
    if os.path.isfile(r+'/'+file_name+'.txt'):
        info_file=open(r+'/'+file_name+'.txt','r')
        info=np.asarray([[float(i) for i in l.split(' ')]  for l in info_file.readlines()])
        print('extracted {0}, shape: {1}'.format(file_name, info.shape))
#         return group.create_dataset(file_name,data=info, compression="gzip", compression_opts=9)
        return group.create_dataset(file_name,data=info, compression="gzip", compression_opts=2)
    else:
        print('could not find {0} in {1}.'.format(file_name,group.name))
        return []

In [13]:
while [ len(runs) != 0 ]:
    r=runs.pop(0)
    print("[{1[2]}/{1[1]}_{1[3]:02d}:{1[4]:02d}]: {0} left".format(len(runs),tuple(time.localtime()[0:5])))
    add_dir(f[os.path.basename(r)],'RGB')
    add_dir(f[os.path.basename(r)],'Depth')
    add_dir(f[os.path.basename(r)],'Depth_predicted')
    add_file(f[os.path.basename(r)],'control_info')
    add_file(f[os.path.basename(r)],'collision_info')
    add_file(f[os.path.basename(r)],'collision_info_3')
    add_file(f[os.path.basename(r)],'collision_info_5')
    add_file(f[os.path.basename(r)],'collision_info_7')
    f.flush()

[10/4_13:28]: 24 left
extracted RGB, shape: (805, 360, 640, 3)
extracted Depth, shape: (805, 360, 640)
could not find Depth_predicted in /00000_esat.
extracted control_info, shape: (1610, 7)
could not find collision_info in /00000_esat.
could not find collision_info_3 in /00000_esat.
could not find collision_info_5 in /00000_esat.
could not find collision_info_7 in /00000_esat.
[10/4_13:30]: 23 left
extracted RGB, shape: (891, 360, 640, 3)
extracted Depth, shape: (891, 360, 640)
could not find Depth_predicted in /00001_esat.
extracted control_info, shape: (1782, 7)
could not find collision_info in /00001_esat.
could not find collision_info_3 in /00001_esat.
could not find collision_info_5 in /00001_esat.
could not find collision_info_7 in /00001_esat.
[10/4_13:31]: 22 left
extracted RGB, shape: (755, 360, 640, 3)
extracted Depth, shape: (755, 360, 640)
could not find Depth_predicted in /00002_esat.
extracted control_info, shape: (1510, 7)
could not find collision_info in /00002_esat.
c

extracted RGB, shape: (759, 360, 640, 3)
extracted Depth, shape: (759, 360, 640)
could not find Depth_predicted in /00022_esat.
extracted control_info, shape: (1518, 7)
could not find collision_info in /00022_esat.
could not find collision_info_3 in /00022_esat.
could not find collision_info_5 in /00022_esat.
could not find collision_info_7 in /00022_esat.
[10/4_14:23]: 1 left
extracted RGB, shape: (962, 360, 640, 3)
extracted Depth, shape: (962, 360, 640)
could not find Depth_predicted in /00023_esat.
extracted control_info, shape: (1924, 7)
could not find collision_info in /00023_esat.
could not find collision_info_3 in /00023_esat.
could not find collision_info_5 in /00023_esat.
could not find collision_info_7 in /00023_esat.
[10/4_14:24]: 0 left
extracted RGB, shape: (1175, 360, 640, 3)
extracted Depth, shape: (1175, 360, 640)
could not find Depth_predicted in /00024_esat.
extracted control_info, shape: (2350, 7)
could not find collision_info in /00024_esat.
could not find collisio

IndexError: pop from empty list

In [71]:
# Request that the HDF5 library flush its buffers to disk.
f.flush()

In [75]:
# Read in hdf5 file to check if everything is there:
f = h5py.File(root_dir+'/data.hdf5', 'r')
print 'Reading data.hdf5'
for g in f.keys():
    print g
    for ds in f[g].keys():
        print '---{0}: {1}'.format(ds, f[g][ds].shape)
f.close()

Reading data.hdf5
00000_canyon
---Depth: (130, 360, 640)
---Depth_predicted: (119, 128, 160)
---RGB: (129, 360, 640, 3)
---collision_info: (129, 2)
---collision_info_3: (129, 2)
---collision_info_5: (129, 2)
---collision_info_7: (129, 2)
---control_info: (258, 7)
00001_canyon
---Depth: (96, 360, 640)
---Depth_predicted: (95, 128, 160)
---RGB: (95, 360, 640, 3)
---collision_info: (95, 2)
---collision_info_3: (95, 2)
---collision_info_5: (95, 2)
---collision_info_7: (95, 2)
---control_info: (191, 7)
